* 提出问题：
1. EdX在线教育平台上什么课程和主题最受欢迎？
2. 人们更喜欢Harvard还是MIT的课？这两个学校的课程分布？
3. 课程的完成度怎么样？用户坚持学习了吗？
4. 课程时长有什么特征，对课程完成度是否有影响？
5. 其他影响课程完成度因素？
6. 全球在学习edX的是什么样的用户？

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df=pd.read_csv('../input/course-study/appendix.csv')
df.head()

In [ ]:
df.shape

字段理解：

一共有290条在线课堂数据，字段信息23个，可大致分为4类：

1、课程相关字段：
Institution（online course holders）课程持有机构
Course number(the unique id of each course) 课程编号ID
Launch date上线日期
Course title课程名称
Instructors讲师
Course subject课程主题
Honor code certificates是否提供认证，有（1）没有（0）
Year课程持续时间

2、数量相关字段：
Total course hours总课程时长小时数
Paticipants学生数
Certified 拿到认证人数
Median hours for certification 拿到认证小时数的中位数
Audited （50% course content accessed）完成 50%课程人数

3、百分比相关字段：
%audited 完成 50%课程人数占比
%certified 认证人数占比
%certified of > 50% course content accessed 认证人数在完成 50%课程人数中占比
%play video 播放视频人数占比
% posted in forum 粘贴到论坛占比
%grade higher than zero分数高于0的百分比

4、学员信息相关字段：
Median age用户年龄中位数
%male男性占比
%female女性占比
%bachelor’s degree or higher用户学士学历或以上占比

****数据清洗****

1. 查看重复值

In [ ]:
df.duplicated().value_counts()

该数据集没有检查到重复值

2. 缺失值

In [ ]:
df.isnull().sum()

Instructors讲师列有一个缺失值，查看该行具体数据**

In [ ]:
df[df['Instructors'].isnull()]

观察该缺失值行数据，发现该行数据可以从Course Number课程编号查找到改讲师姓名进行填充

In [ ]:
df[df['Course Number']=='CS50x']['Instructors']

In [ ]:
df['Instructors'].fillna('David Malan',inplace=True)
df['Instructors'].isnull().sum()

****特征提取构建模型****

1. EdX在线教育平台上什么课程和主题最受欢迎？

In [ ]:
#主题
Subject=df.groupby('Course Subject')['Participants (Course Content Accessed)'].sum().sort_values(ascending=False)
Subject=Subject.to_frame()
Subject.head()

In [ ]:
#课程
course=df.groupby('Course Title')['Participants (Course Content Accessed)'].sum().sort_values(ascending=False)
course=course.to_frame()
course.head()

In [ ]:
plt.figure(figsize=(10,15))
# 可视化主题-子图1
plt.subplot(211)
label=Subject.index
value=Subject['Participants (Course Content Accessed)']
plt.pie(value,labels=label,autopct='%.1f%%',)
plt.title('Course Subject')

# 可视化前10的课程-子图2
plt.subplot(212)
course_top10=course[0:10]
plt.plot(course_top10)
plt.title('top10--Course Title')
plt.xticks(rotation=80)
plt.xlabel('Course Title')
plt.ylabel('学生数')


该edX平台主题只有4类，其中Computer Science这个主题是最受欢迎的；

而Computer Science主题下，排名第一的课程是Introduction to Computer Science，学生数量高达690059，远远超过其他


2. 人们更喜欢Harvard还是MIT的课？这两个学校的开设课程分布？

In [ ]:
#学校课程欢迎度
sc=pd.pivot_table(df,index='Institution',columns='Course Subject',values='Participants (Course Content Accessed)',aggfunc='sum')
sc

In [ ]:
#学校开设课程分布
sc_sub=pd.pivot_table(df,index='Institution',columns='Course Subject',values='Participants (Course Content Accessed)',aggfunc='count')
sc_sub

In [ ]:
plt.figure(figsize=(15,8))
#学校课程学生数-子图1
plt.subplot(121)
index=sc.index
y1=sc['Computer Science']
y2=sc['Government, Health, and Social Science']
y3=sc['Humanities, History, Design, Religion, and Education']
y4=sc['Science, Technology, Engineering, and Mathematics']
plt.bar(index,y1,label='workday')
plt.bar(index,y2,bottom=y1,label='Government, Health, and Social Science')
plt.bar(index,y3,bottom=y1+y2,label='Humanities, History, Design, Religion, and Education')
plt.bar(index,y4,bottom=y1+y2+y3,label='Science, Technology, Engineering, and Mathematics')
plt.title('HarvardX&MITx---Participants')
plt.legend()
#学校开设课程-子图2
x=sc_sub.index
yy1=sc_sub['Computer Science']
yy2=sc_sub['Government, Health, and Social Science']
yy3=sc_sub['Humanities, History, Design, Religion, and Education']
yy4=sc_sub['Science, Technology, Engineering, and Mathematics']
plt.subplot(122)
plt.bar(x,yy1)
plt.bar(x,yy2,bottom=yy1)
plt.bar(x,yy3,bottom=yy1+yy2)
plt.bar(x,yy4,bottom=yy1+yy2+yy3)
plt.title('HarvardX&MITx---Course Subject  ')
plt.legend()



**Harvard&MIT发布课程数**
Harvard发布最多数量的课程是Humanities, History, Design, Religion, and Education（此主题类课程数量达到80） ，只发布了4门Computer Science。Harvard为edX平台贡献了绝大多数人文科学课程。
MIT发布最多的课程是Science, Technology, Engineering, and Mathematics类课程（此主题类课程数量为83），MIT为edX平台提供的课程主要是工科类

**Harvard&MIT课程学生数**
很有意思的是，虽然 harvard 只开设了4门Computer Science类课程（MIT开设了26门此类课），但是学习人数超过MIT,代表Harvard的计算机科学课程非常受欢迎。
另外两所学校发布Government, Health, and Social Science主题类课程数目几乎差不多，不过用户更喜欢MIT的课程。


3. 课程的完成度怎么样？用户坚持学习了吗？


In [ ]:
stu=df[['% Audited','% Certified']]
stu.describe()

In [ ]:
plt.figure(figsize=(10,5))
plt.boxplot([stu['% Audited'],stu['% Certified']])   
plt.xticks([1,2],['% Audited','% Certified']) 
plt.title('课程完成度')


完成50%课程的人数占比在2.63%-83.96%浮动，平均值24.9%.

最后取得课程认证人数占比在0-33.98%浮动，平均值为7.78%。可见最后拿到认证的学生真是少之又少。

4. 课程时长有什么特征，对课程完成度是否有影响？

In [ ]:
df.columns

In [ ]:
play=df[['Course Title','Total Course Hours (Thousands)','Participants (Course Content Accessed)','% Played Video','% Certified']]
play.describe()

In [ ]:
plt.figure(figsize=(10,5))
plt.hist(df['Total Course Hours (Thousands)'],bins=15)  
plt.xlabel('Total Course Hours (Thousands)')

edX平台的课程的时长范围在0.11~895间波动，平均时长在95。在290条数据中，其中就有超过170条数据的时长在100左右。

In [ ]:
cour_play=play.groupby(by='Course Title')['Total Course Hours (Thousands)','% Certified'].mean()
cour_play.head()

In [ ]:
corr=play.corr()
print(corr['% Certified'])
plt.figure(figsize=(10,6))
plt.scatter(cour_play['Total Course Hours (Thousands)'],cour_play['% Certified'])

从相关系数和散点图来看，课程时长和课程完成度呈负相关关系,但相关系数不高，仅仅只有-0.076180，随着播放时长的提高对课程完成度影响并不会特别大

5.其他影响课程完成度因素？


In [ ]:
corr=df.corr()
corr
corr['% Certified'].sort_values()
# import seaborn as sns
# sns.pairplot(df)

从相关性系数来看% Posted in Forum ，Median Age 对课程完成度有影响

6. 全球在学习edX的是什么样的用户？

In [ ]:
1# 年龄特征
df['Median Age'].describe()

In [ ]:
plt.boxplot(df['Median Age'])

用户年龄在22-53岁浮动，edX的用户群体是社会职场人士及即将步入社会的大学生。平均年龄是29岁，这是进入职场3-6年的时期，用户的目标应该在于增加职场技能竞争力。

In [ ]:
#2、性别特征
sex=df[['% Male','% Female']]
sex.describe()

In [ ]:
label=sex.columns  
plt.pie(sex.mean(),labels=label,autopct='%.1f%%')
plt.title('Course Subject')

In [ ]:
df.groupby(by='Course Subject')['% Male','% Female'].mean()

男性学员平均占比67%，女性学员平均33%。edx的用户以男性用户为主，用户性别差异很大。
其中Computer Science类课程男女占比差异性很大；
而Humanities, History, Design, Religion, and Education类课程男女比例确基本平衡。

In [ ]:
#3、学历特征
educational=df["% Bachelor's Degree or Higher"]
educational.describe()

72%的edX用户的学历在学士或学士学位以上，还有28%在学士学历以下。
edX用户大多为学士或学士学位以上,e是社会职场人士及即将步入社会的大学生，目标在于增加职场技能竞争力。

****总结****

1、EdX在线教育平台上，最受欢迎的是计算机科学课（introdution to computer science）。用户们最关注的课程主题是计算机科学。
可以看出计算机科学类课程需求很大，34%的用户们曾学习过计算机科学方面课程。根据用户年龄层次推断，学习计算机课程的原因是为了增加职场中竞争力。

2、Harvard和MIT开设的课程各有千秋，MIT开设了大量科学技术工程类课程，而Harvard开设了大量人文科学类课程，用户们可以根据自己的需要选择课程。
在人文、历史、设计、宗教和教育类课程和 计算机科学课程Harvard更受欢迎，
而科学、技术、工程、数学类课程和政府、卫生和社会科学类课程，MIT更受欢迎。

3、 在线平台学习能坚持学到一半的人不多，最终拿到认证的人更加稀少。
在edX上只有平均24.9%的学生完成一半课程内容，只有平均7.78%的学生最后取得认证。大部分用户没有坚持学习。推测可能跟无人监督，课程难度太高等因素有关，但具体原因需要更多数据进行分析才能得出。

4、edX平台的课程平均时长在95。而随着时长的增加，最后取得认证的学生数也相对减少。

5、edX平台用户平均年龄是29岁，而用户大多是男性职场人士，72%的edX用户的学历在学士或学士学位以上。也有一部分低于学士学历的用户在edX上学习。